# What to do

- make the kernel from https://www.kaggle.com/wolfgangb33r/simple-talkingdata-prediction-xgboost/code

# Setup

## Libraries

In [1]:
import pandas as pd
import git
from pandas import DataFrame, Series
from pandas.api.types import CategoricalDtype

import os
import sys
repo = git.Repo('.', search_parent_directories=True)
module_path = repo.working_tree_dir
print(module_path + "\t<<<<<< get to project")
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
import datetime as dt
from datetime import date
from boto.s3.connection import S3Connection
import tarfile

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


/home/ec2-user/kaggle/kaggle_talkingdata	<<<<<< get to project


In [2]:
#Importing the required libraries
import zipfile
import subprocess
import psutil
import base64
import zlib
import glob
import dask.dataframe as dd
from dask.delayed import delayed
import dask.bag as db
import calendar
import glob
import time
from sys import getsizeof
from boto.s3 import key
from datetime import datetime,timedelta
from boto.s3.key import Key

In [4]:
import numpy as np
import pandas as pd
import math
import time
import os.path

import xgboost as xgb

## Other notebooks

In [6]:
import user_defined_functions.notebookutil as nbu
sys.meta_path.append(nbu.NotebookFinder())

ModuleNotFoundError: No module named 'user_defined_functions'

In [ ]:
import user_defined_functions.helper_functions as hp

# Main Code

In [5]:
start_time = time.time()
# create a xgboost model
model = xgb.XGBClassifier(max_depth=3, n_estimators=600, learning_rate=0.05)

In [ ]:
# already prepared for chunked learning in case that full training data
# set does not fit into memory
# fit the model
chunksize = 4000000 
chunk_idx = 0 
chunk_max = 1
for chunk in pd.read_csv('../input/train.csv', chunksize=chunksize):
    # Extracting new features
    chunk['hour'] = pd.to_datetime(chunk.click_time).dt.hour.astype('uint8')
    chunk['day'] = pd.to_datetime(chunk.click_time).dt.day.astype('uint8')
    train_X = chunk.as_matrix(columns=['ip', 'app', 'device', 'os', 'channel', 'hour', 'day'])
    if chunk_idx > 0: # not load in first run
        model.fit(train_X, chunk['is_attributed'], xgb_model=model)
    else:
        model.fit(train_X, chunk['is_attributed'])
    print("Processed chunk %d" % chunk_idx)
    chunk_idx = chunk_idx + 1
    if chunk_idx >= chunk_max:
        break

In [ ]:

        
    # read test data set
    test = pd.read_csv('../input/test.csv')
    test['hour'] = pd.to_datetime(test.click_time).dt.hour.astype('uint8')
    test['day'] = pd.to_datetime(test.click_time).dt.day.astype('uint8')
    
    test_X = test.as_matrix(columns=['ip', 'app', 'device', 'os', 'channel', 'hour', 'day'])
    start_time = print_duration (start_time, "Finished training, start prediction")   
    # predict the propabilities for binary classes    
    pred = model.predict_proba(test_X)
    
    start_time = print_duration (start_time, "Finished prediction, start store results")    
    submission = pd.read_csv("../input/sample_submission.csv")
    submission['is_attributed'] = pred[:,1]
    submission.to_csv("submission.csv", index=False)
    start_time = print_duration(start_time, "Finished to store result")